In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from google.cloud import bigquery

In [3]:
client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [4]:
#Reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request to fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [5]:
tables = list(client.list_tables(dataset))

In [6]:
#These are the list of the tables in the database
for table in tables:  
    print(table.table_id)

comments
full
full_201510
stories


In [7]:
table_ref = dataset_ref.table("full")
table = client.get_table(table_ref)

In [9]:
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', 'Story title', ()),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", ()),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', ()),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', ()),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ()),
 SchemaField(

In [10]:
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

In [12]:
query_data= client.query(query)

In [14]:
us_city = query_data.to_dataframe()

In [15]:
us_city.city.value_counts().head()

Phoenix-Mesa-Scottsdale                     88
Houston                                     82
Los Angeles-Long Beach-Santa Ana            68
Riverside-San Bernardino-Ontario            60
New York-Northern New Jersey-Long Island    60
Name: city, dtype: int64

In [16]:
query1 = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """
query_data1= client.query(query1)
us_cityfull = query_data1.to_dataframe()
us_cityfull.head()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,Aberdeen,BROWN,US,no2,14.0000,2020-06-11 01:00:00+00:00,µg/m³,DEFRA,45.468597,-98.494064,1.0
1,Aberdeen,BROWN,US,o3,65.0000,2020-06-11 01:00:00+00:00,µg/m³,DEFRA,45.468597,-98.494064,8.0
2,Aberdeen,BROWN,US,pm10,10.0000,2020-06-11 01:00:00+00:00,µg/m³,DEFRA,45.468597,-98.494064,24.0
3,Aberdeen,BROWN,US,pm25,5.0000,2020-06-11 01:00:00+00:00,µg/m³,DEFRA,45.468597,-98.494064,24.0
4,Freeport South Avenue I C1012,Houston,US,so2,0.0001,2016-03-06 18:00:00+00:00,ppm,Texas,28.964430,-95.354830,1.0


In [19]:
table_ref = dataset_ref.table("comments")
table = client.get_table(table_ref)

In [20]:
table.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', 'Unique comment ID', ()),
 SchemaField('by', 'STRING', 'NULLABLE', 'Username of commenter', ()),
 SchemaField('author', 'STRING', 'NULLABLE', 'Username of author', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE', 'Human readable time in UTC (format: YYYY-MM-DD hh:mm:ss)', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Comment text', ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ())]

In [18]:
# Query to select comments that received more than 10 replies
query_popular = """
                SELECT parent, COUNT(id)
                FROM `bigquery-public-data.hacker_news.comments`
                GROUP BY parent
                HAVING COUNT(id) > 10
                """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_popular, job_config=safe_config)
popular = query_job.to_dataframe()
popular.head()

,parent,f0_
0,6855145,39
1,3495062,38
2,6969401,48
3,4349650,44
4,9162566,41


In [21]:
query_all = """
                SELECT parent, COUNT(id)
                FROM `bigquery-public-data.hacker_news.comments`
                GROUP BY parent
                """
query_allcom= client.query(query_all)
query_com = query_allcom.to_dataframe()
query_com.head()

,parent,f0_
0,1701284,17
1,2582717,12
2,182280,10
3,6532985,31
4,4907108,9


In [22]:
table_ref = dataset_ref.table("stories")
table = client.get_table(table_ref)
table.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', 'Unique story ID', ()),
 SchemaField('by', 'STRING', 'NULLABLE', 'Username of submitter', ()),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE', 'Human readable time in UTC (format: YYYY-MM-DD hh:mm:ss)', ()),
 SchemaField('title', 'STRING', 'NULLABLE', 'Story title', ()),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story text', ()),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story descendants', ()),
 SchemaField('author', 'STRING', 'NULLABLE', 'Username of author', ())]

In [ ]:
query_all = """
                SELECT DISTINCT author, COUNT(author)
                FROM `bigquery-public-data.hacker_news.stories`
                GROUP BY author
                """
query_allcom= client.query(query_all)
query_com = query_allcom.to_dataframe()
query_com.head()